In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("dataset/HR.csv")

In [4]:
data.part.unique() #part这一栏的所有出现过的值

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

In [5]:
data.groupby(["salary", "part"]).size()

salary  part       
high    IT               83
        RandD            51
        accounting       74
        hr               45
        management      225
        marketing        80
        product_mng      68
        sales           269
        support         141
        technical       201
low     IT              609
        RandD           364
        accounting      358
        hr              335
        management      180
        marketing       402
        product_mng     451
        sales          2099
        support        1146
        technical      1372
medium  IT              535
        RandD           372
        accounting      335
        hr              359
        management      225
        marketing       376
        product_mng     383
        sales          1772
        support         942
        technical      1147
dtype: int64

In [7]:
data = data.join(pd.get_dummies(data.salary))   #salary这一列转化成one hot编码（将"low" "high"等属性转化成010这种数值）
data = data.join(pd.get_dummies(data.part))
del data["salary"]
del data["part"]

In [9]:
data

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,high,low,...,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.38,0.53,2,157,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14995,0.37,0.48,2,160,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14996,0.37,0.53,2,143,3,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
14997,0.11,0.96,6,280,4,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [10]:
data.left.value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [12]:
#数据预处理
y_data = data.left.values.reshape(-1,1)
Y = torch.from_numpy(y_data).type(torch.FloatTensor)
x_data = data[[c for c in data.columns if c !="left"]].values
X = torch.from_numpy(x_data).type(torch.FloatTensor)

In [13]:
#创建模型
from torch import nn

自定义模型 

nn.module 继承这个类 

__init__ 初始化所有层 

forward 定义模型的运算过程（前向传播的过程）

In [14]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.liner_1 = nn.Linear(20, 64) #输入20个属性，输出64个属性
        self.liner_2 = nn.Linear(64, 64)
        self.liner_3 = nn.Linear(64, 1) #最终输出一个属性
        self.relu = nn.ReLU() #激活层
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.liner_1(input)
        x = self.relu(x) #激活层
        x = self.liner_2(x)
        x = self.relu(x) #激活层
        x = self.liner_3(x)
        x = self.sigmoid(x) #激活层：逻辑回归的输出
        return x

In [15]:
model = Model()